In [1]:
import requests
import numpy as np
import os
from bs4 import BeautifulSoup
import re
import pandas as pd
import zipfile
import pickle

In [2]:
# get the list of ids from 9_way_dataset.zip

with zipfile.ZipFile('9_way_dataset.zip', 'r') as zip_file:
    # Assuming there's only one file inside the ZIP
    filename = zip_file.namelist()[0]
    # Open the file inside the ZIP
    with zip_file.open(filename) as f:
        # Load the pickled object
        data_9 = pickle.load(f)

In [3]:
# get the list of ids from 9_way_dataset
ids = []
for i in range(6,9):
    for data in data_9[i]:
        ids.append(data[0])

ids = list(set(ids))
print(ids)
ids = [int(i) for i in ids]
print(ids)

['08923', '00769', '08133', '29930', '02208', '07001', '03184', '292670', '08383', '302467', '08918', '67720', '03206', '06432', '53032', '167544', '53028', '03193', '08381', '139938', '114194', '77507', '03204', '03192', '100522', '08385', '112920', '08919', '01324', '03154', '112335', '510468', '271048', '05021', '96311', '06407', '77317', '05831', '08380', '00776', '500035', '106565', '04668', '00354', '03275', '61929', '05833', '02187', '00361', '06400', '67709', '12956', '64016', '517076', '08411', '03324', '06416', '03282', '34664', '111926', '08917', '243297', '518119', '475089', '51585', '03326', '06408', '12663', '109061', '22079', '453818', '25151', '05829', '471790', '53761', '05857', '04319', '75864', '86698', '111840', '114991', '04672', '01005', '00772', '56708', '01027', '58174', '04578', '05823', '00763', '302472', '13488', '06085', '15557', '53604', '135170', '00757', '289728', '72279', '12668', '08384', '25071', '08947', '08401', '415887', '98355', '06433', '302230', 

In [5]:
# get pickle file
with open('9_way_pdf_dict.pkl', 'rb') as f:
    ids_9 = pickle.load(f)
    
url_dict={}
not_valid_ids = []
for i in ids:
    try:
        url_dict[i]=ids_9[i]
    except:
        not_valid_ids.append(i)
        continue
print(len(ids))
print(len(url_dict))
print(len(not_valid_ids))

882
802
80


In [9]:
print(not_valid_ids)

[8383, 8381, 8385, 8380, 64016, 517076, 8411, 12663, 75864, 12668, 8384, 8401, 12050, 10498, 1998, 8377, 8408, 417049, 54984, 8407, 12686, 8414, 1987, 8409, 8399, 8378, 12669, 8415, 508537, 64798, 12741, 8387, 8379, 423306, 12735, 8386, 20432, 478334, 12736, 8400, 272949, 12664, 6082, 75281, 8410, 517077, 12734, 517177, 12689, 8372, 204257, 12685, 2002, 64839, 8406, 508540, 78338, 78332, 68891, 12660, 171211, 12666, 12049, 9878, 10914, 12738, 517175, 8413, 12670, 75183, 12048, 8405, 10492, 12662, 10996, 508549, 172555, 78311, 517181, 8374]


In [6]:
for pdf_id, url in url_dict.items():
    print(url, pdf_id)

https://imslp.org/wiki/Adagio_in_C_major,_D.349_(Schubert,_Franz) 8923
https://imslp.org/wiki/Sinfonia_in_F_major,_BWV_794_(Bach,_Johann_Sebastian) 769
https://imslp.org/wiki/Grande_fantaisie_sur_des_motifs_de_Niobe,_S.419_(Liszt,_Franz) 8133
https://imslp.org/wiki/Prelude_and_Fugue_in_C_major,_BWV_870_(Bach,_Johann_Sebastian) 29930
https://imslp.org/wiki/Prelude_and_Fugue_in_C-sharp_major,_BWV_848_(Bach,_Johann_Sebastian) 2208
https://imslp.org/wiki/12_Variations_on_the_Russian_Dance_from_%27Das_Waldm%C3%A4dchen%27,_WoO_71_(Beethoven,_Ludwig_van) 7001
https://imslp.org/wiki/Piano_Sonata_No.23,_Op.57_(Beethoven,_Ludwig_van) 3184
https://imslp.org/wiki/Piano_Sonata_No.11_in_A_major,_K.331/300i_(Mozart,_Wolfgang_Amadeus) 292670
https://imslp.org/wiki/2_St%C3%BCcke_aus_Tannh%C3%A4user_und_Lohengrin,_S.445_(Liszt,_Franz) 302467
https://imslp.org/wiki/Allegretto_in_C_major,_D.346_(Schubert,_Franz) 8918
https://imslp.org/wiki/Hungarian_Rhapsody_No.7,_S.244/7_(Liszt,_Franz) 67720
https://imsl

In [7]:
# functions to get page text and parse out the general information sectioin

def get_page_text(url):
    '''Retrive the text from url.'''
    # parse web content into English text
    try:
        response = requests.get(url)
        response.raise_for_status()  # Check for any request errors

        soup = BeautifulSoup(response.content, 'html.parser')
        # Extract all text from the page, excluding script and style tags
        text = soup.get_text(separator='\n', strip=True)

        return text
    except requests.exceptions.RequestException as e:
        print(f"Error: {e}")
        return None

def extract_text_between_strings_second(text, start_string, end_string):
    '''
    Extracts the substring between the second occurrence of start_string and end_string in text.
    '''
    # Find the second occurrence of the start_string (the second occurence of "general information" gives us our metadata)
    if text:
        start_index = text.find(start_string, text.find(start_string) + len(start_string))
        if start_index == -1:  # If start_string is not found
            return None

        # Find the occurrence of the end_string
        end_index = text.find(end_string, start_index)
        if end_index == -1:  # If end_string is not found
            return None

        # Extract the substring between the second occurrence of start_string and end_string
        extracted_text = text[start_index + len(start_string):end_index].strip()

        return extracted_text

def retrieve_general_information(url):
    '''Retrieves the general information section from the IMSLP page of a musical score until Instrumentation.'''
    # retrieve the general information section
    parsed_text = extract_text_between_strings_second(get_page_text(url),'General Information','Instrumentation')
    return parsed_text

def extract_text_between_strings_new(text, start_string, end_string):
    '''
    Extracts the substring between start_string and end_string in text.
    '''
    start_index = text.find(start_string)
    if start_index == -1:  # If start_string is not found
        return None

    # Find the occurrence of the end_string
    end_index = text.find(end_string, start_index)
    if end_index == -1:  # If end_string is not found
        return None

    # Extract the substring between the second occurrence of start_string and end_string
    extracted_text = text[start_index + len(start_string):end_index].strip()

    return extracted_text

In [10]:
# find general information section

attribute_list = ['Work Title', 'Alt\nernative\n.\nTitle', 'Name Translations', 'Name Aliases', 'Authorities', 'Composer',
                        'Opus/Catalogue Number', 'I-Catalogue Number', 'Key', 'Movements/Sections', 'Year/Date of Composition', 
                        'First Pub\nlication', 'Librettist', 'Language', 'Copyright Information','Dedication', 'Average Duration', 'Composer Time Period', "Piece Style"]
list_of_deletable_headings = ['Opus/Catalogue Number','I-Catalogue Number','Movements/Sections','Year/Date of Composition','First Pub\nlication','Average Duration','Composer Time Period']
bad_heading_pattern = r'^.*\n'

info = {}
for pdf_id, url in url_dict.items():
    parsed_text = retrieve_general_information(url)
    
    if parsed_text:

        info_dic = {}
        
        present_attributes = []
        # check which attributes are present in the parsed text
        for attribute in attribute_list:
            if parsed_text.find(attribute) != -1:
                present_attributes.append(attribute)
        
        # extract the text for each present attribute
        for attribute in attribute_list[:-1]:
            if attribute in present_attributes:
                info_dic[attribute] = extract_text_between_strings_new(parsed_text, attribute, present_attributes[present_attributes.index(attribute) + 1] if present_attributes.index(attribute) != len(present_attributes) - 1 else "Romantic")
                
        info_dic['Piece Style'] = parsed_text[parsed_text.find('Piece Style')+12:] # extract the piece style from the end of the general information section
        info_dic['Instrumentation'] = 'Piano'
        info_dic['url'] = url
        
        if 'Alt\nernative\n.\nTitle' in present_attributes:
            alt_title = info_dic['Alt\nernative\n.\nTitle']
            info_dic['Alternative Title'] = alt_title
            del info_dic['Alt\nernative\n.\nTitle']
        
        if 'First Pub\nlication' in present_attributes:
            alt_title = info_dic['First Pub\nlication']
            info_dic['First Publication'] = alt_title
            del info_dic['First Pub\nlication']
        
        for heading in list_of_deletable_headings:
            if heading in info_dic.keys():
                try:
                    info_dic[heading] = re.sub(bad_heading_pattern, '', info_dic[heading])
                except:
                    pass
        
        info[pdf_id] = info_dic
        
    else:
        print("String not found or invalid input.")

802


In [14]:
# create pickle file from the dictionary
with open('9_way_metadata.pkl', 'wb') as f:
    pickle.dump(info, f)